In [ ]:

import cv2
import os

num_classes = 5  # Number of categories for the dataset
images_per_category = 1000  # Number of images to capture for each category
save_directory = "dataset"  # Directory to store captured images

# Initialize the webcam
webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

# Loop through each category
for category_id in range(num_classes):

    # Create a folder for the current category
    category_folder = os.path.join(save_directory, str(category_id))
    os.makedirs(category_folder, exist_ok=True)

    print(f"Preparing to capture images for category {category_id}. Press 's' to start or 'q' to quit.")
    
    while True:
        # Capture a frame from the webcam
        ret, frame = webcam.read()
        if not ret:
            print("Error: Unable to capture video frame.")
            break

        # Display instructions on the live video feed
        cv2.putText(frame, "Press 's' to start capturing or 'q' to quit", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Webcam Feed", frame)

        # Wait for user input
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s'):
            break  # Start capturing images
        elif key == ord('q'):
            webcam.release()
            cv2.destroyAllWindows()
            exit()  # Quit the program

    # Counter for the number of images captured in the current category
    image_count = 0
    while image_count < images_per_category:
        # Capture a frame from the webcam
        ret, frame = webcam.read()
        if not ret:
            print("Error: Unable to capture video frame.")
            break

        # Display the live video feed
        cv2.imshow("Webcam Feed", frame)

        # Check if the user wants to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            webcam.release()
            cv2.destroyAllWindows()
            exit()

        # Save the captured frame as an image
        image_file = os.path.join(category_folder, f'{image_count}.jpg')
        cv2.imwrite(image_file, frame)
        image_count += 1
        print(f"Saved image {image_count}/{images_per_category}")

# Release resources and close windows
webcam.release()
cv2.destroyAllWindows()




In [ ]:

import pickle
import cv2
import os

embeddings_file_path = r"./landmarks.pkl"


if not os.path.exists(embeddings_file_path):
    print(f"Error: File not found: {embeddings_file_path}")
else:
   
    with open(embeddings_file_path, 'rb') as file:
        embeddings_data = pickle.load(file)

webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Error: Unable to access the webcam.")
else:
    while True:
       
        ret, frame = webcam.read()
        if not ret:
            print("Error: Unable to read the video frame.")
            break

        cv2.putText(frame, "Press 's' to capture", (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1.3, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow("Webcam Feed", frame)

        # Check for user input to capture or quit
        key = cv2.waitKey(25) & 0xFF
        if key == ord('s'):
            break  
        elif key == ord('q'):
            webcam.release()
            cv2.destroyAllWindows()
            exit()  

    
    webcam.release()
    cv2.destroyAllWindows()



In [ ]:

import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Load dataset from landmarks file
landmarks_file = "landmarks.pkl"

with open(landmarks_file, 'rb') as file:
    dataset = pickle.load(file)

features = np.array(dataset['data'])
target_labels = np.array(dataset['labels'])

# Split data into training and testing sets (80% training, 20% testing)
train_features, test_features, train_labels, test_labels = train_test_split(
    features, target_labels, test_size=0.2, random_state=42, stratify=target_labels
)

# Initialize different machine learning models
models = {
    'RandomForest': RandomForestClassifier(),
    'KNeighbors': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000)  # Increased iterations for better convergence
}

# Evaluate each model's accuracy
accuracy_results = {}

for model_name, model_instance in models.items():
    model_instance.fit(train_features, train_labels)
    predictions = model_instance.predict(test_features)
    accuracy = accuracy_score(test_labels, predictions)
    accuracy_results[model_name] = accuracy

best_model_name = max(accuracy_results, key=accuracy_results.get)
best_model_accuracy = accuracy_results[best_model_name]


for model_name, model_instance in models.items():
    model_instance.fit(features, target_labels)
    model_file = f"{model_name}_trained_model.pkl"
    with open(model_file, 'wb') as file:
        pickle.dump(model_instance, file)
    print(f"Model saved: {model_name} -> {model_file}")


print(f"The best model is '{best_model_name}' with an accuracy of {best_model_accuracy:.2f}")



In [ ]:

import mediapipe as mp
import numpy as np
import pickle
import cv2



model_filepath = 'LogisticRegression_model.pkl'
with open(model_filepath, 'rb') as model_file:
    trained_model = pickle.load(model_file)


webcam = cv2.VideoCapture(0)

hand_tracker = mp.solutions.hands.Hands(
    static_image_mode=False,
    min_detection_confidence=0.2
)
drawing_utils = mp.solutions.drawing_utils

letter_labels = {index: chr(index + 65) for index in range(26)}

while True:
    
    hand_landmarks_data = []
    x_coords = []
    y_coords = []

 
    ret, frame = webcam.read()
    if not ret:
        print("Error: Could not capture frame.")
        break

    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the hand landmarks in the current frame
    processed_frame = hand_tracker.process(frame_rgb)

    if processed_frame.multi_hand_landmarks:
        for hand_landmarks in processed_frame.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                x_coords.append(landmark.x)
                y_coords.append(landmark.y)

            # Normalize coordinates and store data for prediction
            for landmark in hand_landmarks.landmark:
                hand_landmarks_data.append(landmark.x - min(x_coords))  # Normalize x
                hand_landmarks_data.append(landmark.y - min(y_coords))  # Normalize y
                hand_landmarks_data.append(landmark.z)  # Z coordinate (no normalization)

            # Draw the hand landmarks on the frame
            drawing_utils.draw_landmarks(
                frame,
                hand_landmarks,
                mp.solutions.hands.HAND_CONNECTIONS,
                drawing_utils.DrawingSpec(color=(0, 255, 0), thickness=4),
                drawing_utils.DrawingSpec(color=(0, 0, 255), thickness=2)
            )

        # Determine the bounding box for the hand on the frame
        min_x = int(min(x_coords) * frame.shape[1]) - 10
        max_x = int(max(x_coords) * frame.shape[1]) + 10
        
        min_y = int(min(y_coords) * frame.shape[0]) - 10
        max_y = int(max(y_coords) * frame.shape[0]) + 10

        # Check if we have all 63 features needed for the model
        if len(hand_landmarks_data) == 63:
            # Predict the letter based on the landmarks data
            prediction = trained_model.predict([np.asarray(hand_landmarks_data)])
            predicted_letter = letter_labels[int(prediction[0])]  # Map prediction to letter

            # Draw a bounding box and display the predicted letter on the frame
            cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), (255, 255, 255), 2)
            cv2.putText(frame, predicted_letter, (min_x, min_y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display the frame with the predicted letter
    cv2.imshow('Sign Language Detection', frame)
 # Break the loop if the user presses 'e'
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break


webcam.release()
cv2.destroyAllWindows()



In [38]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

# Load the trained best model
model = pickle.load(open('KNeighbors_model.pkl', 'rb'))

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.3)

# Label dictionary for A-Z
labels_dict = {i: chr(i + 65) for i in range(26)}

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                x = lm.x
                y = lm.y
                z = lm.z  # Include the z-coordinate
                x_.append(x)
                y_.append(y)

            for lm in hand_landmarks.landmark:
                data_aux.append(lm.x - min(x_))  # Normalize x-coordinate
                data_aux.append(lm.y - min(y_))  # Normalize y-coordinate
                data_aux.append(lm.z)  # Include the z-coordinate without normalization

            # Draw the hand landmarks on the frame
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), circle_radius=2)
            )

        # Get the bounding box for the hand
        x1 = int(min(x_) * W) - 10
        x2 = int(max(x_) * W) + 10
        y1 = int(min(y_) * H) - 10
        y2 = int(max(y_) * H) + 10

        if len(data_aux) == 63:  # Ensure there are 63 features for prediction
            # Predict the class of the hand sign using the best model
            pred = model.predict([np.asarray(data_aux)])
            pred_char = labels_dict[int(pred[0])]  # Get the predicted character

            # Draw a rectangle and display the predicted character
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
            cv2.putText(frame, pred_char, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the frame with the predicted character
    cv2.imshow('Sign Detected', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [39]:

import pickle
import cv2
import mediapipe as mp
import numpy as np

# Load the pre-trained model for gesture recognition
model_path = 'RandomForest_model.pkl'
with open(model_path, 'rb') as model_file:
    gesture_model = pickle.load(model_file)

# Initialize the webcam for capturing frames
webcam_capture = cv2.VideoCapture(0)

# Initialize Mediapipe hand tracking
hand_tracking = mp.solutions.hands.Hands(
    static_image_mode=False,
    min_detection_confidence=0.3
)
drawing_utils = mp.solutions.drawing_utils

# Define the label mapping for gestures (A-Z)
gesture_labels = {index: chr(index + 65) for index in range(26)}

while True:
    # Initialize data containers for hand landmarks and coordinates
    normalized_data = []
    x_coordinates = []
    y_coordinates = []

    # Capture a frame from the webcam
    success, frame = webcam_capture.read()
    if not success:
        print("Failed to capture the frame. Exiting.")
        break

    # Get the height and width of the frame and convert it to RGB for Mediapipe processing
    frame_height, frame_width, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to detect hand landmarks
    processed_frame = hand_tracking.process(rgb_frame)
    if processed_frame.multi_hand_landmarks:
        for landmarks in processed_frame.multi_hand_landmarks:
            
            for landmark in landmarks.landmark:
                x_coordinates.append(landmark.x)
                y_coordinates.append(landmark.y)

           
            for landmark in landmarks.landmark:
                normalized_data.append(landmark.x - min(x_coordinates))  # Normalize x
                normalized_data.append(landmark.y - min(y_coordinates))  # Normalize y
                normalized_data.append(landmark.z)  # Keep z as is

            # Draw the landmarks on the frame
            drawing_utils.draw_landmarks(
                frame,
                landmarks,
                mp.solutions.hands.HAND_CONNECTIONS,
                drawing_utils.DrawingSpec(color=(0, 255, 0), circle_radius=4),
                drawing_utils.DrawingSpec(color=(0, 0, 255), circle_radius=2)
            )

        # Calculate the bounding box for the hand
        x_min = int(min(x_coordinates) * frame_width) - 10
        x_max = int(max(x_coordinates) * frame_width) + 10
        y_min = int(min(y_coordinates) * frame_height) - 10
        y_max = int(max(y_coordinates) * frame_height) + 10

        # Ensure there are 63 features for prediction
        if len(normalized_data) == 63:
            # Predict the gesture based on the features using the trained model
            gesture_prediction = gesture_model.predict([np.asarray(normalized_data)])
            predicted_gesture = gesture_labels[int(gesture_prediction[0])]  # Map the prediction to a letter

            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (255, 255, 255), 2)
            cv2.putText(frame, predicted_gesture, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('Gesture Recognition', frame)

    
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release the webcam and close the window
webcam_capture.release()
cv2.destroyAllWindows()
